In [ ]:
import glob
import os
import warnings
import mlflow
import mlflow.pytorch 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [ ]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [ ]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")


In [ ]:
# Defining small sample vs. end smaple
whole_data_set = False
# end sample or small sample    
if whole_data_set == True:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=420, random_state=RSEED)
    test_df = test_df.sample(n=420, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))

Calss definition dataloader (do not change over models)

Step 1: U-Net Model and Data Preparation

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from PIL import Image
import pydicom

class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        dicom_image = pydicom.dcmread(img_path)
        image_array = dicom_image.pixel_array
        image = Image.fromarray(image_array)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        x = torch.tensor(self.data.iloc[idx]['x'], dtype=torch.float32)
        y = torch.tensor(self.data.iloc[idx]['y'], dtype=torch.float32)

        return image, torch.tensor([x, y]) 

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),  # Ensures single channel output
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Example values for grayscale images
])

# Create the dataset
dataset = MRILocalizationDataset(data=train_df, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 32
num_epochs = 10
learning_rate = 0.0001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(in_channels=1, out_channels=2)  # Single channel input for grayscale images

model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

experiment_name = "MRI_Localization_UNet"
mlflow.set_experiment(experiment_name)

# Define an input example for MLflow
input_example = torch.randn(1, 3, 224, 224, device=device)  # Random tensor simulating one sample image

with mlflow.start_run():
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("model_architecture", "U-Net for Localization")
    mlflow.log_param("output_coordinates", "x, y")

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for images, coordinates in train_loader:
            images, coordinates = images.to(device), coordinates.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            # Flatten the output tensor to match the target shape (batch_size, 2)
            outputs = outputs.view(outputs.size(0), -1)

            # Ensure the shape of outputs matches the target (x, y)
            if outputs.size(1) != 2:
                outputs = outputs[:, :2]  # Only take the first 2 channels if the output size is incorrect

            loss = criterion(outputs, coordinates)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        mlflow.log_metric("train_loss", train_loss, step=epoch)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, coordinates in val_loader:
                images, coordinates = images.to(device), coordinates.to(device)
                outputs = model(images)

                # Flatten the output tensor to match the target shape (batch_size, 2)
                outputs = outputs.view(outputs.size(0), -1)
                if outputs.size(1) != 2:
                    outputs = outputs[:, :2]

                loss = criterion(outputs, coordinates)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        mlflow.log_metric("val_loss", val_loss, step=epoch)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Log the model with input_example
    input_example_np = input_example.cpu().numpy()  # Convert tensor to numpy array
    mlflow.pytorch.log_model(model, "model", input_example=input_example_np)
mlflow.pytorch.log_model(model, "model", input_example=input_example)

    # Plot and save the loss curves
    plt.figure(figsize=(10, 5))
    plt.plot(range(num_epochs), train_losses, label='Train Loss')
    plt.plot(range(num_epochs), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig("loss_curve.png")
    mlflow.log_artifact("loss_curve.png")


predictions

In [ ]:
train_df

test predicion

In [ ]:

test_df = test_df.drop(['severity', 'condition', 'level', 'series_id', 'missing_image'], axis=1)


In [ ]:
print(test_df.to_string(index=False, header=True)) 

predicting

In [ ]:
test_data = test_df

In [ ]:
import numpy as np
import pandas as pd
import torch
import mlflow
from torch.utils.data import DataLoader

# Load the trained model from MLflow
model = mlflow.pytorch.load_model("runs:/ec2baecd43734618aedf3e465aec6693/model")
model.eval()  # Set the model to evaluation mode

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Ensure the model is on the right device

# Model outputs a heatmap instead of coordinates
predicted_coords = []
with torch.no_grad():  # Disable gradient computation for inference
    for images, _ in test_loader:  # Unpack images and ignore labels
        images = images.to(device)
        heatmaps = model(images)  # Get model output as heatmap
        heatmaps = heatmaps.cpu().numpy()  # Move to CPU for processing
        
        for heatmap in heatmaps:
            # Find the brightest point (peak) in the heatmap
            y, x = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
            predicted_coords.append((x, y))  # Store (x, y) coordinates of the peak

# Save predicted coordinates along with original test data
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)
test_df_with_preds.to_csv("test_with_predicted_coordinates.csv", index=False)

print("Inference complete. Predicted coordinates saved.")


In [ ]:
import numpy as np
import pandas as pd
import torch

from torch.utils.data import DataLoader

model = mlflow.pytorch.load_model("runs:/ec2baecd43734618aedf3e465aec6693/model")
model.eval()

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model outputs a heatmap instead of coordinates
predicted_coords = []
with torch.no_grad():  # Disable gradient computation for inference
    for images, _ in test_loader:  # Unpack images and ignore labels
        images = images.to(device)
        heatmaps = model(images)  # Get model output as heatmap
        heatmaps = heatmaps.cpu().numpy()  # Move to CPU for processing
        
        for heatmap in heatmaps:
            # Find the brightest point (peak) in the heatmap
            y, x = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
            predicted_coords.append((x, y))

# Save predicted coordinates along with original test data
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)
test_df_with_preds.to_csv("test_with_predicted_coordinates.csv", index=False)

print("Inference complete. Predicted coordinates saved.")


In [ ]:
# Load the model from MLflow
from torch.utils.data import DataLoader

model = mlflow.pytorch.load_model("runs:/ec2baecd43734618aedf3e465aec6693/model")
model.eval()

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Inference on new test data
predicted_coords = []
with torch.no_grad():  # Disable gradient computation for inference
    for images, _ in test_loader:  # Unpack images and ignore labels
        images = images.to(device)
        outputs = model(images)  # Get model output
        outputs = outputs.view(outputs.size(0), -1)  # Reshape to (batch_size, 2) for x, y coordinates
        predicted_coords.extend(outputs.cpu().numpy())  # Move to CPU and add to results

# Save predicted coordinates
import pandas as pd
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])
output_df.to_csv("predicted_coordinates.csv", index=False)

print("Inference complete. Predicted coordinates saved.")


In [ ]:
import numpy as np
import pandas as pd
import torch
import mlflow
from torch.utils.data import DataLoader

# Load the trained model from MLflow
model = mlflow.pytorch.load_model("runs:/ec2baecd43734618aedf3e465aec6693/model")
model.eval()  # Set the model to evaluation mode

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Ensure the model is on the right device

# Inference on new test data
predicted_coords = []
with torch.no_grad():  # Disable gradient computation for inference
    for images, _ in test_loader:  # Unpack images and ignore labels
        images = images.to(device)
        outputs = model(images)  # Get model output
        outputs = outputs.view(outputs.size(0), -1)  # Reshape to (batch_size, 2) for x, y coordinates
        predicted_coords.extend(outputs.cpu().numpy())  # Move to CPU and add to results

# Convert the predicted coordinates to a DataFrame
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])

# Combine original test_df with predicted coordinates
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)

# Save the resulting dataframe to a CSV file
test_df_with_preds.to_csv("test_with_predicted_coordinates.csv", index=False)

print("Inference complete. Predicted coordinates saved.")


In [ ]:
test_df_with_preds